In [3]:
def ADCDecode(message):
    try:
        rawmsg = int(message.hex(), 16)
        #print(message.hex())
    except ValueError:
        return 0,0,0,0,0
    else:
        #print(hex(rawmsg))
        msgIndex = (rawmsg >> 8*8) & 0xFF
        ADC2 = (rawmsg >> 8*6) & 0xFFFF
        ADC1 = (rawmsg >> 8*4) & 0xFFFF
        temperature1 = (rawmsg >> 20) & 0xFFF
        temperature2 = (rawmsg >> 8) & 0xFFF
    return msgIndex, ADC1, ADC2, temperature1, temperature2

def ADCConvert(ADC1, ADC2):
    Vref = 2.5
    ADCmax = 0xFFF0
    
    V1 = (2 * ADC1 - ADCmax) * Vref /ADCmax
    V2 = -(2 * ADC2 - ADCmax) * Vref /ADCmax
    return V1, V2

raw = b'\x03\x0c \t\x18\x04\x00 \n'
msgIndex, ADC1, ADC2, temperature1, temperature2 = ADCDecode(raw)
print(msgIndex, ADC1, ADC2, temperature1, temperature2)

    #print(msgIndex)
    #print("ADC1", hex(ADC1))
    #print("ADC2", hex(ADC2))
    #print("Temp1", hex(temperature1))
    #print("Temp2", hex(temperature2))

3 2328 3104 64 32


In [10]:
# got this Readline from https://github.com/pyserial/pyserial/issues/216#issuecomment-369414522, so i guess it works faster??
class ReadLine:
    def __init__(self, s):
        self.buf = bytearray()
        self.s = s

    def readline(self):
        i = self.buf.find(b"\n")
        if i >= 0:
            r = self.buf[:i+1]
            self.buf = self.buf[i+1:]
            return r
        while True:
            i = max(1, min(2048, self.s.in_waiting))
            data = self.s.read(i)
            i = data.find(b"\n")
            if i >= 0:
                r = self.buf + data[:i+1]
                self.buf[0:] = data[i+1:]
                return r
            else:
                self.buf.extend(data)

def setADCRate(ADCRate):
    cmd = "R"
    cmd += str(ADCRate)
    bytecmd = bytearray(cmd.encode())
    ser.write(bytecmd)
    print(ser.readline().decode())

def setVoffset(channel, voltage):
    cmd = "O"
    cmd += str(channel)
    cmd += ","
    cmd += str(int(voltage)*1000)
    bytecmd = bytearray(cmd.encode())
    ser.write(bytecmd)
    print(ser.readline().decode())

def measureR0(current):
    cmd = "M1,"
    cmd += str(current)
    bytecmd = bytearray(cmd.encode())
    ser.write(bytecmd)
    print(ser.readline().decode())

def stopMeasureR0():
    cmd = "M0"
    bytecmd = bytearray(cmd.encode())
    ser.write(bytecmd)
    print(ser.readline().decode())


In [15]:
import serial
import serial.tools.list_ports
import time


#print("Available ports:")
#print([comport.device for comport in serial.tools.list_ports.comports()])

COMPort = 'COM5'
#serialBaudRate = 921600
serialBaudRate = 1152000
serialInterCharTimeout = 100e-3 # 100us 

print("Using serial port",COMPort, "at",serialBaudRate, "baud" )
try:
    ser = serial.Serial(COMPort, baudrate=serialBaudRate, bytesize=8, parity='N', stopbits=1, timeout=1)
    print("Opened")
except Exception as error:
    print("Error:", error)
    raise

ser.flushInput()
ser.flushOutput()
rl = ReadLine(ser)

setADCRate(1000)
# Start ADC capture
#ser.write(b'S')



# Get ADC zero

ser.write(b'A')
Vzero = 0
Izero = 0
for i in range(20):
    message = ser.read(9)
    msgIndex, ADC1, ADC2, temperature1, temperature2 = ADCDecode(message)
    #print(msgIndex, ADC1, ADC2, temperature1, temperature2)
    V, I = ADCConvert(ADC1, ADC2)
    Vzero += V
    Izero += I
    print (V,I)
ser.write(b's')

Vzero /= 20
Izero /= 20
print ("Vzero",Vzero,"Izero",Izero)

time.sleep(1)

ser.flushInput()
ser.flushOutput()

# Start R0 at 100mA
ser.write(b'M1,200')

#print(rl.readline().decode())
print(ser.readline().decode())
for i in range(15):
    #print(rl.readline().decode())
    #message = ADCDecode(ser.readline())
    #message = ser.readline()#.decode()
    message = ser.read(9)
    #print(message)
    msgIndex, ADC1, ADC2, temperature1, temperature2 = ADCDecode(message)
    print(msgIndex, ADC1, ADC2, temperature1, temperature2)
    V, I = ADCConvert(ADC1, ADC2)
    print ("V:", round(V-Vzero, 4), "I:", round(I-Izero, 4), "R0:", round((V-Vzero)/(I-Izero), 3) )
    print()
    #print(V/I)

# Stop R0 measurement
ser.write(b'M0')

# Stop ADC capture
ser.write(b's')
ser.close()
print("Stopped")

Using serial port COM5 at 1152000 baud
Error: module 'serial' has no attribute 'Serial'


AttributeError: module 'serial' has no attribute 'Serial'